<a href="https://colab.research.google.com/github/AmayaDes/Neuro-Symbolic_PCOS-Detection-FYP/blob/main/notebooks/03_PCOS_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Authenticate
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download
!kaggle datasets download -d amayajayarathna/pcos-ultrasound-dataset-trainvaltest-split

!unzip pcos-ultrasound-dataset-trainvaltest-split.zip


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/amayajayarathna/pcos-ultrasound-dataset-trainvaltest-split
License(s): unknown
 86% 127M/148M [00:00<00:00, 1.33GB/s]
100% 148M/148M [00:00<00:00, 1.21GB/s]
Archive:  pcos-ultrasound-dataset-trainvaltest-split.zip
  inflating: PCOS_Dataset_Split/test/normal/Dataset-1_data_train_img1.jpeg  
  inflating: PCOS_Dataset_Split/test/normal/Dataset-1_data_train_img_0_1084.jpg  
  inflating: PCOS_Dataset_Split/test/normal/Dataset-1_data_train_img_0_110.jpg  
  inflating: PCOS_Dataset_Split/test/normal/Dataset-1_data_train_img_0_117.jpg  
  inflating: PCOS_Dataset_Split/test/normal/Dataset-1_data_train_img_0_1181.jpg  
  inflating: PCOS_Dataset_Split/test/normal/Dataset-1_data_train_img_0_130.jpg  
  inflating: PCOS_Dataset_Split/test/normal/Dataset-1_data_train_img_0_1408.jpg  
  inflating: PCOS_Dataset_Split/test/normal/Dataset-1_data_train_img_0_1494.jpg  
  inflating: PCOS_Dataset_Split/test/normal/Dataset-1_data_

In [2]:
"""
PCOS Feature Extraction

Classes:
- PCOSImage: Represents a single ultrasound image
- FeatureExtractor: Handles feature computation
- PCOSDataset: Manages collection of images
- FeatureVisualizer: Handles visualization and statistics
"""

import os
import pandas as pd
import numpy as np
import cv2
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern, blob_log
from scipy import ndimage, stats
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import joblib

# ========================
# CONFIGURATION
# ========================

ORGANIZED_DIR = "/content/PCOS_Dataset_Split"

# ========================
# LOAD METADATA FROM GITHUB
# ========================

base_url = "https://raw.githubusercontent.com/AmayaDes/Neuro-Symbolic_PCOS-Detection-FYP/main/results/"

train_metadata_df = pd.read_csv(base_url + "train_metadata.csv")
val_metadata_df = pd.read_csv(base_url + "val_metadata.csv")
test_metadata_df = pd.read_csv(base_url + "test_metadata.csv")

print(f" Loaded metadata from GitHub:")
print(f" Train: {len(train_metadata_df)} images")
print(f" Val:   {len(val_metadata_df)} images")
print(f" Test:  {len(test_metadata_df)} images")


def fix_image_path(row, base_dir, split):
    """Update path to local downloaded location"""
    return os.path.join(base_dir, split, row['label_name'], row['filename'])

train_metadata_df['image_path'] = train_metadata_df.apply(
    lambda row: fix_image_path(row, ORGANIZED_DIR, 'train'),
    axis=1
)

val_metadata_df['image_path'] = val_metadata_df.apply(
    lambda row: fix_image_path(row, ORGANIZED_DIR, 'val'),
    axis=1
)

test_metadata_df['image_path'] = test_metadata_df.apply(
    lambda row: fix_image_path(row, ORGANIZED_DIR, 'test'),
    axis=1
)

# Verify first image exists
sample_path = train_metadata_df.iloc[0]['image_path']
print(f" Sample path: {sample_path}")
print(f" Exists: {os.path.exists(sample_path)}")

if not os.path.exists(sample_path):
    raise FileNotFoundError(f" Image not found. Check ORGANIZED_DIR path: {ORGANIZED_DIR}")


# ========================
# CLASS 1: PCOSImage
# ========================

class PCOSImage:
    """
    Represents a single PCOS ultrasound image

    Attributes:
        image_path (str): Path to image file
        image_id (str): Unique identifier
        label (int): 0=Normal, 1=PCOS
        label_name (str): 'normal' or 'pcos'
        raw_image (numpy.ndarray): Loaded image
        preprocessed_image (numpy.ndarray): Processed image
        features (dict): Extracted features
    """

    def __init__(self, image_path, image_id, label, label_name):
        """Initialize PCOSImage object"""
        self.image_path = image_path
        self.image_id = image_id
        self.label = label
        self.label_name = label_name
        self.raw_image = None
        self.preprocessed_image = None
        self.features = {}

    def load_image(self):
        """Load image from file"""
        self.raw_image = cv2.imread(self.image_path, cv2.IMREAD_GRAYSCALE)
        if self.raw_image is None or self.raw_image.size == 0:
            raise ValueError(f"Cannot load image: {self.image_path}")
        return self

    def preprocess(self):
        """Preprocess image for feature extraction"""
        # Despeckle filtering
        denoised = cv2.fastNlMeansDenoising(self.raw_image, h=10)
        # Resize to standard size
        self.preprocessed_image = cv2.resize(denoised, (224, 224))
        return self

    def get_features_dict(self):
        """Get features as dictionary with metadata"""
        result = {
            'ImageID': self.image_id,
            'label': self.label,
            'Binary_Label': self.label_name,
            'image_path': self.image_path
        }
        result.update(self.features)
        return result


# ========================
# CLASS 2: FeatureExtractor
# ========================

class FeatureExtractor:
    """
    Extracts PCOS-specific features from ultrasound images

    FEATURES:
    1. Stromal echogenicity - Ovarian brightness
    2. Follicle count - LoG blob detection
    3. Follicle density - Normalized count
    4. Average follicle size - Mean blob radius
    5. Peripheral distribution - Pearl necklace sign
    6. Ovarian area index - Ovarian size
    7. Ovarian circularity - Shape irregularity
    8. Texture heterogeneity - Tissue irregularity
    9. Speckle index - Ultrasound noise patterns
    """

    def __init__(self, pcos_image):
        """
        Initialize FeatureExtractor

        Args:
            pcos_image (PCOSImage): Image object to extract features from
        """
        self.image = pcos_image.preprocessed_image
        self.pcos_image = pcos_image
        self.features = {}

    def extract_all_features(self):
        """Extract all PCOS features"""
        self.compute_stromal_echogenicity()
        self.compute_follicle_pattern_blob()
        self.compute_peripheral_distribution()
        self.compute_ovarian_morphology()
        self.compute_texture_heterogeneity()
        self.compute_speckle_index()
        self.compute_additional_features()

        # Store in PCOSImage object
        self.pcos_image.features = self.features
        return self.features

    def compute_stromal_echogenicity(self):
        """
        FEATURE 1: Stromal Echogenicity
        """
        self.features['stromal_echogenicity'] = np.mean(self.image)
        self.features['stromal_std'] = np.std(self.image)

    def compute_follicle_pattern_blob(self):
        """
        FEATURES 2-4: Follicle Detection using Blob Detection
        Uses Laplacian of Gaussian (LoG) blob detection
        """

        # Normalize image to [0, 1] for blob_log
        img_normalized = self.image.astype(np.float32) / 255.0

        # Detect dark circular blobs (follicles)
        blobs = blob_log(
            img_normalized,
            min_sigma=3,
            max_sigma=8,
            num_sigma=8,
            threshold=0.130,
            overlap=0.3
        )

        # Feature 2: Absolute follicle count
        follicle_count = len(blobs)
        self.features['follicle_count'] = follicle_count

        # Feature 3: Follicle density (normalized by image area)
        image_area = self.image.shape[0] * self.image.shape[1]
        self.features['follicle_density'] = (follicle_count / image_area) * 10000

        # Feature 4: Average follicle size
        if follicle_count > 0:
            radii = blobs[:, 2] * np.sqrt(2)
            self.features['avg_follicle_size'] = np.mean(radii)
            self.features['follicle_size_std'] = np.std(radii)
        else:
            self.features['avg_follicle_size'] = 0
            self.features['follicle_size_std'] = 0

        # Hough Circles
        circles = cv2.HoughCircles(
            self.image, cv2.HOUGH_GRADIENT,
            dp=1.5, minDist=15,
            param1=100, param2=20,
            minRadius=3, maxRadius=12
        )
        self.features['hough_circle_count'] = len(circles[0]) if circles is not None else 0

        # EDGE DENSITY
        edges = cv2.Canny(self.image, 50, 150)
        self.features['edge_density'] = np.sum(edges > 0) / edges.size

    def compute_peripheral_distribution(self):
        """
        FEATURE 5: Peripheral Distribution
        """
        h, w = self.image.shape
        center = (w // 2, h // 2)

        # Create center mask
        center_mask = np.zeros_like(self.image, dtype=np.uint8)
        cv2.circle(center_mask, center, min(h, w) // 4, 255, -1)

        # Calculate brightness in regions
        center_region = self.image[center_mask > 0]
        peripheral_region = self.image[center_mask == 0]

        center_brightness = np.mean(center_region) if len(center_region) > 0 else 1
        peripheral_brightness = np.mean(peripheral_region) if len(peripheral_region) > 0 else 1

        self.features['peripheral_distribution'] = peripheral_brightness / (center_brightness + 1e-6)

    def compute_ovarian_morphology(self):
        """
        FEATURES 6-7: Ovarian Morphology
        Clinical: PCOS ovaries are enlarged and irregular
        """
        # Adaptive thresholding
        blur = cv2.GaussianBlur(self.image, (5, 5), 0)
        binary = cv2.adaptiveThreshold(
            blur, 255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY_INV,
            11, 2
        )

        # Morphological cleaning
        kernel = np.ones((5, 5), np.uint8)
        binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

        # Find contours
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            area = cv2.contourArea(largest_contour)
            perimeter = cv2.arcLength(largest_contour, True)

            # Circularity (1.0 = perfect circle, <1.0 = irregular)
            circularity = 4 * np.pi * area / (perimeter**2 + 1e-6)

            self.features['ovarian_area_index'] = area
            self.features['ovarian_circularity'] = circularity
        else:
            self.features['ovarian_area_index'] = 0
            self.features['ovarian_circularity'] = 0

    def compute_texture_heterogeneity(self):
        """
        FEATURE 8: Texture Heterogeneity
        """
        # Quantize image to 64 levels
        img_q = (self.image / 4).astype(np.uint8)

        # GLCM texture analysis
        glcm = graycomatrix(
            img_q,
            distances=[1, 2],
            angles=[0, np.pi/4, np.pi/2, 3*np.pi/4],
            levels=64,
            symmetric=True,
            normed=True
        )

        contrast = graycoprops(glcm, 'contrast').mean()
        dissimilarity = graycoprops(glcm, 'dissimilarity').mean()
        self.features['texture_heterogeneity'] = (contrast + dissimilarity) / 2.0

    def compute_speckle_index(self):
        """
        FEATURE 9: Speckle Index
        """
        # Calculate local standard deviation (speckle measure)
        local_std = ndimage.generic_filter(self.image, np.std, size=5)
        self.features['speckle_index'] = np.mean(local_std)

    def compute_additional_features(self):
        """Compute supplementary texture features"""
        # Entropy
        hist, _ = np.histogram(self.image, bins=256, range=(0, 256))
        hist = hist / (hist.sum() + 1e-10)
        self.features['entropy'] = -np.sum(hist * np.log2(hist + 1e-10))

        # Local Binary Pattern
        lbp = local_binary_pattern(self.image, P=8, R=1, method='uniform')
        self.features['lbp_mean'] = np.mean(lbp)
        self.features['lbp_std'] = np.std(lbp)

        # Intensity variance
        self.features['intensity_variance'] = np.var(self.image)


# ========================
# CLASS 3: PCOSDataset
# ========================

class PCOSDataset:
    """
    Manages a collection of PCOS images (train/val/test split)

    Attributes:
        split_name (str): 'train', 'val', or 'test'
        organized_dir (str): Path to organized data directory
        images (list): List of PCOSImage objects
        metadata_df (DataFrame): Original metadata
        features_df (DataFrame): Extracted features
    """

    def __init__(self, split_name, organized_dir, metadata_df):
        """Initialize PCOSDataset with pre-loaded metadata"""
        self.split_name = split_name
        self.organized_dir = organized_dir
        self.metadata_df = metadata_df  # Use provided metadata
        self.images = []
        self.features_df = None

    def load_from_metadata(self):
        """Load images from provided metadata DataFrame"""
        print(f"\n{'='*70}")
        print(f"LOADING {self.split_name.upper()} DATASET")
        print(f"{'='*70}")
        print(f"Total images: {len(self.metadata_df)}")
        print(f"  PCOS:   {(self.metadata_df['label']==1).sum()}")
        print(f"  Normal: {(self.metadata_df['label']==0).sum()}")

        # Create PCOSImage objects
        for _, row in self.metadata_df.iterrows():
            img_obj = PCOSImage(
                image_path=row['image_path'],
                image_id=row['filename'] if 'filename' in row else f"img_{_}",
                label=row['label'],
                label_name=row['label_name']
            )
            self.images.append(img_obj)

        return self

    def extract_features_batch(self):
        """Extract features from all images in dataset"""
        print(f"\n{'='*70}")
        print(f"EXTRACTING FEATURES: {self.split_name.upper()}")
        print(f"{'='*70}")

        all_features = []
        failed_count = 0

        for img_obj in tqdm(self.images, desc=f"Processing {self.split_name}", ncols=70):
            try:
                # Load and preprocess
                img_obj.load_image().preprocess()

                # Validation check
                if img_obj.preprocessed_image is None:
                    print(f"\n Skipping {img_obj.image_id}: preprocessing failed")
                    failed_count += 1
                    continue

                # Extract features
                extractor = FeatureExtractor(img_obj)
                extractor.extract_all_features()

                # Collect results
                all_features.append(img_obj.get_features_dict())

            except Exception as e:
                print(f"\n Error processing {img_obj.image_id}: {str(e)}")
                failed_count += 1

        if failed_count > 0:
            print(f"\n Failed: {failed_count} images")

        # Create DataFrame
        self.features_df = pd.DataFrame(all_features)

        # Reorder columns
        metadata_cols = ['ImageID', 'label', 'Binary_Label', 'image_path']
        feature_cols = [col for col in self.features_df.columns if col not in metadata_cols]
        self.features_df = self.features_df[metadata_cols + feature_cols]

        print(f"\n Extracted features: {len(self.features_df)} images")
        print(f" Feature count: {len(feature_cols)}")

        return self

    def save_features(self):
        """Save features to CSV"""
        output_path = os.path.join(self.organized_dir, f'features_{self.split_name}.csv')
        self.features_df.to_csv(output_path, index=False)
        print(f" Saved: {output_path}")
        return self


# ========================
# CLASS 4: FeatureVisualizer
# ========================

class FeatureVisualizer:
    """
    Handles visualization and statistical analysis of PCOS features
    """

    def __init__(self, organized_dir):
        """Initialize FeatureVisualizer"""
        self.organized_dir = organized_dir
        self.train_df = None
        self.scaler = None

        self.pcos_features = [
            'stromal_echogenicity',
            'follicle_count',
            'follicle_density',
            'avg_follicle_size',
            'peripheral_distribution',
            'ovarian_area_index',
            'ovarian_circularity',
            'texture_heterogeneity',
            'speckle_index'
        ]

    def load_training_features(self):
        """Load training features for visualization"""
        features_path = os.path.join(self.organized_dir, 'features_train.csv')

        if not os.path.exists(features_path):
            raise FileNotFoundError(f"Training features not found: {features_path}")

        self.train_df = pd.read_csv(features_path)
        print(f"\n Loaded {len(self.train_df)} training feature vectors")
        return self

    def normalize_features(self):
        """Z-score normalization"""
        print("\n" + "="*70)
        print("APPLYING Z-SCORE NORMALIZATION")
        print("="*70)

        # Fit scaler on training data
        self.scaler = StandardScaler()
        self.train_df[self.pcos_features] = self.scaler.fit_transform(
            self.train_df[self.pcos_features]
        )

        # Save scaler for later use
        scaler_path = os.path.join(self.organized_dir, "feature_scaler.pkl")
        joblib.dump(self.scaler, scaler_path)

        print(f" Features normalized (mean=0, std=1)")
        print(f" Scaler saved: feature_scaler.pkl")

        # Save normalized training features
        self.train_df.to_csv(
            os.path.join(self.organized_dir, 'features_train_normalized.csv'),
            index=False
        )
        print(f" Saved: features_train_normalized.csv")

        return self

    def apply_normalization_to_splits(self):
        """Apply fitted scaler to validation and test sets"""
        print("\n Applying normalization to val and test sets...")

        for split in ['val', 'test']:
            df_path = os.path.join(self.organized_dir, f'features_{split}.csv')

            if not os.path.exists(df_path):
                print(f" Skipping {split}: file not found")
                continue

            df = pd.read_csv(df_path)

            # Apply scaler (fitted on train)
            df[self.pcos_features] = self.scaler.transform(df[self.pcos_features])

            # Save normalized features
            output_path = os.path.join(self.organized_dir, f'features_{split}_normalized.csv')
            df.to_csv(output_path, index=False)
            print(f" {split}: features_{split}_normalized.csv")

        return self

    def plot_feature_distributions(self):
        """Plot feature distributions: Normal vs PCOS"""
        print("\n Creating feature distribution plots...")

        # ORIGINAL features for visualization
        original_train = pd.read_csv(os.path.join(self.organized_dir, 'features_train.csv'))

        # Creating grid for all features
        n_features = len(self.pcos_features)
        n_cols = 3
        n_rows = (n_features + n_cols - 1) // n_cols

        fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, n_rows * 5))
        axes = axes.ravel()

        for idx, feature in enumerate(self.pcos_features):
            normal = original_train[original_train['label'] == 0][feature]
            pcos = original_train[original_train['label'] == 1][feature]

            axes[idx].hist(normal, bins=30, alpha=0.6, label='Normal', color='blue', edgecolor='black')
            axes[idx].hist(pcos, bins=30, alpha=0.6, label='PCOS', color='red', edgecolor='black')
            axes[idx].set_title(f'{feature}', fontsize=12, fontweight='bold')
            axes[idx].set_xlabel('Value')
            axes[idx].set_ylabel('Frequency')
            axes[idx].legend()
            axes[idx].grid(alpha=0.3)

            # Mean lines
            axes[idx].axvline(normal.mean(), color='blue', linestyle='--', linewidth=2, alpha=0.8)
            axes[idx].axvline(pcos.mean(), color='red', linestyle='--', linewidth=2, alpha=0.8)

            # Statistics text
            normal_mean = normal.mean()
            pcos_mean = pcos.mean()
            diff_pct = ((pcos_mean - normal_mean) / (abs(normal_mean) + 1e-6)) * 100

            axes[idx].text(0.05, 0.95,
                          f'Normal: {normal_mean:.2f}\nPCOS: {pcos_mean:.2f}\nΔ: {diff_pct:+.1f}%',
                          transform=axes[idx].transAxes,
                          verticalalignment='top',
                          bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
                          fontsize=9)

        # Hide extra subplots
        for idx in range(len(self.pcos_features), len(axes)):
            axes[idx].axis('off')

        plt.suptitle('PCOS Feature Distributions: Normal vs PCOS\n(Optimized Blob Detection: threshold=0.130, Cohen\'s d=2.691)',
                     fontsize=14, fontweight='bold')
        plt.tight_layout()

        plot_path = os.path.join(self.organized_dir, 'pcos_feature_distributions.png')
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        print(f" Saved: {plot_path}")
        plt.close()

    def plot_feature_correlations(self):
        """Plot correlation heatmaps"""
        print(" Creating correlation heatmaps...")

        # Use original features
        original_train = pd.read_csv(os.path.join(self.organized_dir, 'features_train.csv'))

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

        normal_corr = original_train[original_train['label']==0][self.pcos_features].corr()
        pcos_corr = original_train[original_train['label']==1][self.pcos_features].corr()

        sns.heatmap(normal_corr, annot=True, fmt='.2f', cmap='Blues', ax=ax1, cbar_kws={'label': 'Correlation'},
                    xticklabels=[f.replace('_', '\n') for f in self.pcos_features],
                    yticklabels=[f.replace('_', '\n') for f in self.pcos_features])
        ax1.set_title('Feature Correlation - Normal', fontsize=12, fontweight='bold')

        sns.heatmap(pcos_corr, annot=True, fmt='.2f', cmap='Reds', ax=ax2, cbar_kws={'label': 'Correlation'},
                    xticklabels=[f.replace('_', '\n') for f in self.pcos_features],
                    yticklabels=[f.replace('_', '\n') for f in self.pcos_features])
        ax2.set_title('Feature Correlation - PCOS', fontsize=12, fontweight='bold')

        plt.tight_layout()

        plot_path = os.path.join(self.organized_dir, 'pcos_feature_correlations.png')
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        print(f" Saved: {plot_path}")
        plt.close()

    def cohens_d(self, x, y):
        """Calculate Cohen's d effect size"""
        nx, ny = len(x), len(y)
        pooled_std = np.sqrt(((nx-1)*x.var() + (ny-1)*y.var()) / (nx+ny-2))
        return (x.mean() - y.mean()) / pooled_std

    def generate_statistical_comparison(self):
        """Generate statistical comparison table with Cohen's d"""
        print("\n" + "="*70)
        print("STATISTICAL COMPARISON: Normal vs PCOS")
        print("="*70)

        # Use original features for statistics
        original_train = pd.read_csv(os.path.join(self.organized_dir, 'features_train.csv'))

        comparison_data = []

        for feature in self.pcos_features:
            normal_vals = original_train[original_train['label']==0][feature]
            pcos_vals = original_train[original_train['label']==1][feature]

            normal_mean = normal_vals.mean()
            pcos_mean = pcos_vals.mean()
            diff_pct = ((pcos_mean - normal_mean) / (abs(normal_mean) + 1e-6)) * 100

            # T-test
            t_stat, p_value = stats.ttest_ind(normal_vals, pcos_vals)

            # Cohen's d effect size
            effect_size = self.cohens_d(normal_vals, pcos_vals)

            # Effect magnitude
            if abs(effect_size) >= 0.8:
                effect_mag = 'Large'
            elif abs(effect_size) >= 0.5:
                effect_mag = 'Medium'
            else:
                effect_mag = 'Small'

            comparison_data.append({
                'Feature': feature,
                'Normal_Mean': normal_mean,
                'Normal_Std': normal_vals.std(),
                'PCOS_Mean': pcos_mean,
                'PCOS_Std': pcos_vals.std(),
                'Change_%': diff_pct,
                'P_Value': p_value,
                'Cohens_d': effect_size,
                'Effect_Magnitude': effect_mag,
                'Significant': '***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else 'ns'
            })

            print(f"\n{feature}:")
            print(f"  Normal: {normal_mean:.3f} (±{normal_vals.std():.3f})")
            print(f"  PCOS:   {pcos_mean:.3f} (±{pcos_vals.std():.3f})")
            print(f"  Change: {diff_pct:+.1f}%")
            print(f"  P-value: {p_value:.4f} {comparison_data[-1]['Significant']}")
            print(f"  Cohen's d: {effect_size:.3f} ({effect_mag})")

        comparison_df = pd.DataFrame(comparison_data)
        csv_path = os.path.join(self.organized_dir, 'feature_comparison.csv')
        comparison_df.to_csv(csv_path, index=False)
        print(f"\n Saved: {csv_path}")

        return comparison_df

    def analyze_feature_importance(self):
        """Random Forest feature importance analysis"""
        print("\n Feature importance analysis...")

        # Use normalized features for ML analysis
        X = self.train_df[self.pcos_features]
        y = self.train_df['label']

        rf = RandomForestClassifier(n_estimators=100, random_state=42)
        rf.fit(X, y)

        importance_df = pd.DataFrame({
            'feature': self.pcos_features,
            'importance': rf.feature_importances_
        }).sort_values('importance', ascending=False)

        print("\n Feature Importance:")
        print(importance_df.to_string(index=False))

        # Plot
        plt.figure(figsize=(10, 8))
        colors = plt.cm.RdYlGn(importance_df['importance'] / importance_df['importance'].max())
        plt.barh(importance_df['feature'], importance_df['importance'], color=colors, edgecolor='black')
        plt.xlabel('Importance', fontsize=12)
        plt.title('PCOS Feature Importance (Random Forest)\nOptimized Blob Detection (threshold=0.130)',
                 fontsize=14, fontweight='bold')
        plt.grid(axis='x', alpha=0.3)
        plt.tight_layout()

        plot_path = os.path.join(self.organized_dir, 'pcos_feature_importance.png')
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        print(f" Saved: {plot_path}")
        plt.close()

        return importance_df


# ========================
# MAIN EXECUTION
# ========================

if __name__ == "__main__":
    print("="*70)
    print("PCOS FEATURE EXTRACTION")
    print("="*70)

    # Create dictionary of metadata dataframes
    metadata_dfs = {
        'train': train_metadata_df,
        'val': val_metadata_df,
        'test': test_metadata_df
    }

    # Process all splits
    for split in ['train', 'val', 'test']:
        try:
            dataset = PCOSDataset(split, ORGANIZED_DIR, metadata_dfs[split])
            dataset.load_from_metadata()
            dataset.extract_features_batch()
            dataset.save_features()
        except Exception as e:
            print(f"\n Error processing {split}: {str(e)}")
            import traceback
            traceback.print_exc()
            continue

    # Visualization and analysis
    try:
        visualizer = FeatureVisualizer(ORGANIZED_DIR)
        visualizer.load_training_features()

        # Normalize features
        visualizer.normalize_features()
        visualizer.apply_normalization_to_splits()

        # Generate visualizations and statistics
        visualizer.plot_feature_distributions()
        visualizer.plot_feature_correlations()
        comparison_df = visualizer.generate_statistical_comparison()
        importance_df = visualizer.analyze_feature_importance()

    except Exception as e:
        print(f"\n Error in visualization: {str(e)}")
        import traceback
        traceback.print_exc()

    # Summary
    print("\n" + "="*70)
    print(" FEATURE EXTRACTION COMPLETE!")
    print("="*70)

    print(f"\n Created files:")
    print(f" features_train.csv (raw features)")
    print(f" features_val.csv (raw features)")
    print(f" features_test.csv (raw features)")
    print(f" features_train_normalized.csv (z-score normalized)")
    print(f" features_val_normalized.csv (z-score normalized)")
    print(f" features_test_normalized.csv (z-score normalized)")
    print(f" feature_scaler.pkl (for deployment)")
    print(f" pcos_feature_distributions.png")
    print(f" pcos_feature_correlations.png")
    print(f" pcos_feature_importance.png")
    print(f" feature_comparison.csv (with Cohen's d)")

 Loaded metadata from GitHub:
 Train: 2783 images
 Val:   596 images
 Test:  597 images
 Sample path: /content/PCOS_Dataset_Split/train/pcos/Dataset-1_data_train_img_0_4883.jpg
 Exists: True
PCOS FEATURE EXTRACTION

LOADING TRAIN DATASET
Total images: 2783
  PCOS:   1162
  Normal: 1621

EXTRACTING FEATURES: TRAIN


Processing train:  61%|█████▍   | 1700/2783 [1:01:25<39:30,  2.19s/it]


 Error processing Dataset-1_data_test_img_0_7.jpg: Cannot load image: /content/PCOS_Dataset_Split/train/normal/Dataset-1_data_test_img_0_7.jpg


Processing train: 100%|█████████| 2783/2783 [1:40:05<00:00,  2.16s/it]



 Failed: 1 images

 Extracted features: 2782 images
 Feature count: 17
 Saved: /content/PCOS_Dataset_Split/features_train.csv

LOADING VAL DATASET
Total images: 596
  PCOS:   249
  Normal: 347

EXTRACTING FEATURES: VAL


Processing val: 100%|███████████████| 596/596 [21:25<00:00,  2.16s/it]



 Extracted features: 596 images
 Feature count: 17
 Saved: /content/PCOS_Dataset_Split/features_val.csv

LOADING TEST DATASET
Total images: 597
  PCOS:   249
  Normal: 348

EXTRACTING FEATURES: TEST


Processing test: 100%|██████████████| 597/597 [21:38<00:00,  2.17s/it]



 Extracted features: 597 images
 Feature count: 17
 Saved: /content/PCOS_Dataset_Split/features_test.csv

 Loaded 2782 training feature vectors

APPLYING Z-SCORE NORMALIZATION
 Features normalized (mean=0, std=1)
 Scaler saved: feature_scaler.pkl
 Saved: features_train_normalized.csv

 Applying normalization to val and test sets...
 val: features_val_normalized.csv
 test: features_test_normalized.csv

 Creating feature distribution plots...
 Saved: /content/PCOS_Dataset_Split/pcos_feature_distributions.png
 Creating correlation heatmaps...
 Saved: /content/PCOS_Dataset_Split/pcos_feature_correlations.png

STATISTICAL COMPARISON: Normal vs PCOS

stromal_echogenicity:
  Normal: 52.768 (±13.380)
  PCOS:   71.909 (±17.232)
  Change: +36.3%
  P-value: 0.0000 ***
  Cohen's d: -1.267 (Large)

follicle_count:
  Normal: 21.830 (±7.808)
  PCOS:   47.115 (±9.847)
  Change: +115.8%
  P-value: 0.0000 ***
  Cohen's d: -2.900 (Large)

follicle_density:
  Normal: 4.351 (±1.556)
  PCOS:   9.390 (±1.96